In [41]:
from __future__ import print_function
import numpy as np
import pylab as plt

from odynn.circuit import Circuit
from odynn import datas
from odynn.neuron import PyBioNeuron


from ipywidgets import interact

p = PyBioNeuron.default_params
dt = 20.

@interact(G=0.2, mdp=-60, scale=2., E=-80, Gba=0.2, mdp2=-60, scale2=2., E2=-80)
def dual(Gab=0.2, mdpab=-60, scaleab=2., Eab=-80, Gba=0.2, mdpba=-60, scaleba=2., Eba=-80):
    sab = {
    'G': Gab,
    'mdp': mdpab,
    'scale': scaleab,
    'E': Eab
    }
    sba = {
    'G': Gba,
    'mdp': mdpba,
    'scale': scaleba,
    'E': Eba
    }
    connections = {(0,1) : sab, (1,0) : sba}
    t = np.arange(0.0, 5000., dt)
    i0 = 20. *((t>900)&(t<1050))
    i1 = 10.*((t>500)&(t<750)) #+ 20.*((t>700)&(t<750)) + 14*((t>1100)&(t<1800))+ 22*((t>1900)&(t<2000))
    i_injs = np.array([i0, i1]).transpose()
    circuit = Circuit(PyBioNeuron([p,p], dt=dt), synapses=connections)
    states, curs = circuit.calculate(i_injs)
    plt.subplot(211)
    plt.plot(states[:,0])
    plt.subplot(212)
    plt.plot(i_injs)
    plt.show()
    

interactive(children=(FloatSlider(value=0.2, description='Gab', max=0.6000000000000001, min=-0.2), IntSlider(v…